# Metaclasses and Attributes

- [Item 29: Use Plain Attributes Instead of Get and Set Methods](#Item-29:-Use-Plain-Attributes-Instead-of-Get-and-Set-Methods)
- [Item 30: Consider @property Instead of Refactoring Attributes](#Item-30:-Consider-@property-Instead-of-Refactoring-Attributes)
- [Item 31: Use Descriptors for Reusable @property Methods](#Item-31:-Use-Descriptors-for-Reusable-@property-Methods)

## Item 29: Use Plain Attributes Instead of Get and Set Methods

In [ ]:
# Bad practice
# Using setters and getters is not Pythonic
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms


class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

r1 = Resistor(50e3)
r1.ohms = 10e2

r1.ohms += 5e2


class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
        
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)


class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms
        

# r3 = BoundedResistance(-5)


class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("can't set attribute")
        self._ohms = ohms
        
r4 = FixedResistance(1e3)
print(f'r4 ohms: {r4.ohms}')
# r4.ohms = 2e3  # Exception


# Don't set other attributes in getter property methods
class MysteriousResistor(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms
    
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After: %5r' % r7.voltage)

### Things to Remember

- Define new class interfaces using simple public attributes, and avoid set and get methods.
- Use *@property* to define special behavior when attributes are accessed on your objects, if necessary.
- Follow the rule of least surprise and avoid weird side effects in your *@property* methods.
- Ensure that *@property* methods are fast; do slow or complex work using normal methods.

## Item 30: Consider *@property* Instead of Refactoring Attributes

In [ ]:
from datetime import datetime, timedelta

class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return ('Bucket(max=quota=%d, quota_consumed=%d)' %
                (self.max_quota, self.quota_consumed))
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed = delta
            
    
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period.delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount
    

def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True


bucket = Bucket(100)
print('Initial', bucket)
bucket.quota = 100
print('Fill', bucket)
bucket.quota -= 30
print('deduct', bucket)
bucket.quota -= 20
print('deduct', bucket)
bucket.quota -= 50
print('deduct', bucket)
bucket.quota += 500
print('deduct', bucket)

## Item 31: Use Descriptors for Reusable *@property* Methods


In [ ]:
# Wrong implementation
class Grade(object):
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value
        
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First', first_exam.writing_grade, 'is wrong')

In [ ]:
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        # Problem: Memory leaks
        # self._value = {}
        
        # Improvement: Use WeakKeyDictionary
        self._value = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._value.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value[instance] = value
        
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First', first_exam.writing_grade, 'is right')

### Things to Remember

- Reuse the behavior and validation of @property methods by defining your own descriptor classes.
- Use **WeakKeyDictionary** to ensure that your descriptor classes don't cause memory leaks.
- Don't get bogged down trying to understand exactly how \_\_getattribute__ uses the descriptor protocol for getting and setting attributes.